We will be ingesting, transforming, storing, transforming, and displaying some data.

### 1. First, we'll need data

ImageNet (http://imagenet.stanford.edu/) is a commonly used dataset in machine learning research. We'll be using its taxonomy system.

When you go to http://imagenet.stanford.edu/synset?wnid=n02486410, you'll see a tree on the left. Your job is to get this tree and transform it into a linear form, like this:

```
[
    {name: 'ImageNet 2011 Fall Release', size: 32326},
    {name: 'ImageNet 2011 Fall Release > plant, flora, plant life', size: 4486},
    {name: 'ImageNet 2011 Fall Release > plant, flora, plant life > phytoplankton', size: 2},
    {name: 'ImageNet 2011 Fall Release > plant, flora, plant life > phytoplankton > planktonic algae', size: 0},
    {name: 'ImageNet 2011 Fall Release > plant, flora, plant life > phytoplankton > diatom', size: 0},
    {name: 'ImageNet 2011 Fall Release > plant, flora, plant life > microflora', size: 0},
    ...
]
```

We'll use `>` as a separator of categories/subcategories.

You can get all the data you need here https://s3.amazonaws.com/static.operam.com/assignment/structure_released.xml

### 2. Second, we'll need to store it somewhere

Create a database (use any database system you like or want to try) to store these tuples `(string, number)` and fill it with the data you obtained in the first step.

### 3. Making sense of it

Next, we'll convert it back to a tree. Like this:

```
{
    name: 'ImageNet 2011 Fall Release',
    size: 32326,
    children: [
        {
            name: 'plant, flora, plant life',
            size: 4486,
            children: [
                {
                    name: 'phytoplankton',
                    size: 2,
                    children: [
                        ...
                    ]
                },
                ...
            ]
        },
        ...
    ]
}
```

* Can you write an algorithm that will output such a tree? No cheating here, you have to read this data in a linear form from the database.
* What is the complexity of your algorithm (in big O notation)?

### 3. Now it's time to show the data again

* Can you design and build an interface to show this data?
* Can you implement search in this UI?

<hr>

Use React for frontend, JS/Python for backend and any other frameworks or libraries. If you prefer to use something else, please let me know.


In [ ]:
import xml.etree.cElementTree as ET

file_path = "structure_released.xml"
tree = ET.parse(file_path)
root = tree.getroot()

In [ ]:
# Not sure why the number differ, but there are 60942 synsets in the document.
# base = root.findall(".//synset")
# len(base)

base = root.find(".synset")

In [ ]:
def crawl(synset):
    children = [crawl(child) for child in synset.getchildren()]
    if children:
        return ((synset.attrib['words'], sum([count for _, count, _ in children]) + len(children), children))
    else:
        return (synset.attrib['words'], 0, children)

structure_with_counts = crawl(base)
structure_with_counts

In [ ]:
def flatten(items, prefix = ''):
    for name, count, children in items:
        prefixed_name = f'{prefix} > {name}' if prefix else name
        yield {'name': prefixed_name, 'size': count}
        if children:
            yield from flatten(children, prefix = prefixed_name)

flattened_structure = list(flatten([structure_with_counts]))
flattened_structure

In [ ]:
import pickle

with open('synsets.pickle', 'wb') as f:
    pickle.dump(flattened_structure, f)

In [2]:
import pickle

with open('synsets.pickle', 'rb') as f:
    flattened_structure = pickle.load(f)

In [15]:
def first_true(iterable, default=False, pred=None):
    """Returns the first true value in the iterable.

    If no true value is found, returns *default*

    If *pred* is not None, returns the first item
    for which pred(item) is true.

    """
    # first_true([a,b,c], x) --> a or b or c or x
    # first_true([a,b], x, f) --> a if f(a) else b if f(b) else x
    return next(filter(pred, iterable), default)

def update_nested_dict(nested_dict, keys, size):
    cur = nested_dict
    for path_item in keys[1:]:
        find_result = first_true(cur['children'], pred = lambda child: child['name'] == path_item)
        if find_result:
            cur = find_result
        else:
            cur['children'].append({'name': path_item, 'size': size,'collapsed': True, 'children': []})
            

def reducer(value, element):
    keys = element['name'].split(' > ')

    if len(keys) == 1:
        return {
            'name': element['name'],
            'size': element['size'],
            'collapsed': True,
            'children': []
        }
    
    update_nested_dict(value, keys, element['size'])
    return value

import cProfile

def do_cprofile(func):
    def profiled_func(*args, **kwargs):
        profile = cProfile.Profile()
        try:
            profile.enable()
            result = func(*args, **kwargs)
            profile.disable()
            return result
        finally:
            profile.print_stats()
    return profiled_func
    
from functools import reduce

@do_cprofile
def generate_tree():
    return reduce(reducer, flattened_structure, {})

tree = generate_tree()

         35738965 function calls in 18.896 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    60941    0.448    0.000   18.466    0.000 <ipython-input-15-d9b28130d567>:17(update_nested_dict)
 34711270    8.047    0.000    8.047    0.000 <ipython-input-15-d9b28130d567>:20(<lambda>)
    60942    0.240    0.000   18.825    0.000 <ipython-input-15-d9b28130d567>:27(reducer)
   361604    0.487    0.000   17.998    0.000 <ipython-input-15-d9b28130d567>:4(first_true)
        1    0.000    0.000   18.896   18.896 <ipython-input-15-d9b28130d567>:57(generate_tree)
        1    0.071    0.071   18.896   18.896 {built-in method _functools.reduce}
    60942    0.014    0.000    0.014    0.000 {built-in method builtins.len}
   361604    9.463    0.000   17.510    0.000 {built-in method builtins.next}
    60717    0.020    0.000    0.020    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'di

### Performace

I think that complexity of this algorhytm is n^3. Not really sure about it. However after looking into profiling results I think that best way to improve performance would be to use lookup table to avoid scanning entire tree.

In [16]:
import json

with open('synsets.json', 'w') as f:
    json.dump(tree, f, indent = 2)